Initially thought of estimating forecastability using Approximate Entropy,Sample Entropy and the Average Demand Interval(ADI),square of coefficient of variation(CV^2) methods but the hint provided directed me towards using the features of the Time Series.

 And since, the Prophet model has to be used for anomaly detection, I generated synthetic data for the time series using the Prophet model and then decided to extract the features from the generated data and compare it with the provided data for calculating the forecastability score.

Feature Vector Generation :  

In [1]:
import pandas as pd
import numpy as np
import nolds
from scipy.stats import skew, kurtosis, entropy,variation
from statsmodels.tsa.stattools import adfuller, acf, pacf,kpss
from arch.unitroot import PhillipsPerron
import statsmodels.api as sm

def featureVector(json):
    df= pd.read_json(json)
    df['point_timestamp'] = pd.to_datetime(df['point_timestamp'])
    df['point_value'] = pd.to_numeric(df['point_value'])
    
    #Features
    feature_vector = {}
    feature_vector['SampEn'] = nolds.sampen(df['point_value']) #Sample Entropy
    feature_vector['Hurst'] = nolds.hurst_rs(df['point_value']) #Hurst Exponent
    feature_vector['dfa'] = nolds.dfa(df['point_value']) #Detrended Fluctuation Analysis
    feature_vector['Skew'] = skew(df['point_value']) #Skewness
    feature_vector['Kurt'] = kurtosis(df['point_value']) #Kurtosis
    feature_vector['Entropy'] = entropy(df['point_value']) #Entropy
    time_intervals = df['point_value'].diff().dropna()
    feature_vector['ADI'] = time_intervals.mean() #Average Demand Interval
    cv=df['point_value'].std()/df['point_value'].mean()
    feature_vector['cv2'] = cv**2   # Coefficient of Variation
    feature_vector['ADF'] = adfuller(df['point_value'])[1] #Augmented Dickey-Fuller Test
    feature_vector['ACF'] = acf(df['point_value'])[1] #Auto Correlation
    feature_vector['PACF'] = pacf(df['point_value'])[1] #Partial Auto Correlation
    feature_vector['KPSS'] = kpss(df['point_value'])[1] #Kwiatkowski-Phillips-Schmidt-Shin Test
    feature_vector['Stationary'] = bool(feature_vector['ADF'] <= 0.05 and feature_vector['KPSS'] > 0.05) #Stationarity Check
    feature_vector['PP'] = PhillipsPerron(df['point_value']).pvalue#Phillips-Perron Test
    feature_vector['Normality'] = bool(sm.stats.diagnostic.normal_ad(df['point_value'])[1] > 0.05) #Normality Check
    decompose = sm.tsa.seasonal_decompose(df['point_value'], model='additive', period=6) #Seasonal Decomposition -Half yearly since few time series given have less records 
    trend = decompose.trend.dropna()
    seasonal = decompose.seasonal.dropna()
    residual = decompose.resid.dropna()
    seasonal = decompose.seasonal.dropna()
    feature_vector['Trend Strenth'] = max(0,variation(residual)/(variation(trend)+variation(residual))) #Trend Strength
    feature_vector['Seasonal Strength'] = max(0,variation(residual)/(variation(seasonal)+variation(residual))) #Seasonal Strength
    rolling_mean = df['point_value'].rolling(window=7).mean()
    rolling_std = df['point_value'].rolling(window=7).std()
    feature_vector['Rolling Mean'] = rolling_mean.mean() #Rolling Mean
    feature_vector['Rolling Std'] = rolling_std.mean() #Rolling Standard Deviation
    for key in feature_vector.keys():
        if feature_vector[key] == float('inf') or feature_vector[key] == float('-inf') or feature_vector[key] == float('nan'):
            feature_vector[key] = 0
    return feature_vector


In [15]:
def forecastability_score_euclidean(feature_vector1,feature_vector2):
    # performing euclidean distance between the two feature vectors and return the score in the range [0,10]
    score = 0
    for key in feature_vector1.keys():
        score += (feature_vector1[key]-feature_vector2[key])**2
        print(key,feature_vector1[key],feature_vector2[key])
    return 10/(1+score)


In [2]:
from sklearn.metrics.pairwise import cosine_similarity

def forecastability_score_cosine(feature_vector1, feature_vector2):
    # find cosine similarity between 2 lists given as parameters
    #reshape to 2d
    feature_vector1 = np.array(feature_vector1).reshape(1,-1)
    feature_vector2 = np.array(feature_vector2).reshape(1,-1)
    similarity = cosine_similarity(feature_vector1,feature_vector2)[0][0]
    similarity = 5*(similarity+1)
    return similarity

def forcastability_score_cosine_with_rep(feature_vector):
    representative=pd.read_json('/home/codit/PycharmProjects/DataGenie-Hackathon/Prophet Data/representative.json',typ='series')
    representative=pd.DataFrame(representative)
    rep=representative.iloc[:,0].tolist()
    rep = np.array(rep).reshape(1,-1)
    feature_vector = np.array(feature_vector).reshape(1,-1)
    similarity = cosine_similarity(rep,feature_vector)[0][0]
    similarity = 5*(similarity+1)
    return similarity

In [17]:


def findrepresentative():
    df=[]
    for i in range(5000):
        f=featureVector('/home/codit/PycharmProjects/DataGenie-Hackathon/Prophet Data/sample_'+str(i)+'.json')
        df.append(f)
    df=pd.DataFrame(df)
    return df.mean()   # making the mean to be the representative feature vector for the synthetic Prophet data



In [ ]:

representative=findrepresentative()


Added the representative found to the representative.json file in the prophet data repository.
Now towards finding the threshold for the forecastability score. 

In [4]:
representative

NameError: name 'representative' is not defined

In [3]:
representative.to_json('/home/codit/PycharmProjects/DataGenie-Hackathon/Prophet Data/representative.json')

NameError: name 'representative' is not defined

In [5]:
representative=pd.read_json('/home/codit/PycharmProjects/DataGenie-Hackathon/Prophet Data/representative.json',typ='series')
representative=pd.DataFrame(representative)
rep=representative.iloc[:,0].tolist()

forcastabilility score for the sample time series data    

In [6]:
import os
def findForcastabilityScore():
    scores=[]
    representative=rep
    hourly = os.listdir("/home/codit/Documents/Time Series JSON/hourly")
    daily = os.listdir("/home/codit/Documents/Time Series JSON/daily")
    weekly = os.listdir("/home/codit/Documents/Time Series JSON/weekly")
    monthly = os.listdir("/home/codit/Documents/Time Series JSON/monthly")
    for i in hourly:
        f=list(featureVector("/home/codit/Documents/Time Series JSON/hourly/"+i).values())
        scores.append(forecastability_score_cosine(representative,f))
    for i in daily:
        f=list(featureVector("/home/codit/Documents/Time Series JSON/daily/"+i).values())
        scores.append(forecastability_score_cosine(representative,f))
    for i in weekly:
        f=list(featureVector("/home/codit/Documents/Time Series JSON/weekly/"+i).values())
        scores.append(forecastability_score_cosine(representative,f))
    for i in monthly:
        f=list(featureVector("/home/codit/Documents/Time Series JSON/monthly/"+i).values())
        scores.append(forecastability_score_cosine(representative,f))
    return scores

In [ ]:
sampleforecastScores=findForcastabilityScore()

Approach to calculate threshold is to iterate over the score with a step value and find the score that provides with least RMSE with the provided forecastability score.

In [8]:
import math
from prophet import Prophet
from sklearn.metrics import root_mean_squared_error,mean_absolute_percentage_error


def forecastabilityThreshold(sampleforecastScores):
    sampleforecastScores=pd.Series(sampleforecastScores)
    step = sampleforecastScores.diff().abs().mean()
    start = math.floor(sampleforecastScores.median())
    end = math.ceil(sampleforecastScores.max())
    thresholdRange = np.arange(start,end,step)
    threshold=0
    minMAPEGen = 100000000000
    visited={}
    for i in thresholdRange:
        rmse=[]
        hourly = os.listdir("/home/codit/Documents/Time Series JSON/hourly")
        daily = os.listdir("/home/codit/Documents/Time Series JSON/daily")
        weekly = os.listdir("/home/codit/Documents/Time Series JSON/weekly")
        monthly = os.listdir("/home/codit/Documents/Time Series JSON/monthly")
        
        for j in hourly:
            if "hourly"+j in list(visited.keys()) and visited["hourly"+j]>=i:
                rmse.append(visited["hourly"+j])
            else:
                f=list(featureVector("/home/codit/Documents/Time Series JSON/hourly/"+j).values())
                if forcastability_score_cosine_with_rep(f)>=i:
                    visited["hourly"+j]=predict(pd.read_json("/home/codit/Documents/Time Series JSON/hourly/"+j))
                    rmse.append(visited["hourly"+j])
                    
        for j in daily:
            if "daily"+j in list(visited.keys()) and visited["daily"+j]>=i:
                rmse.append(visited["daily"+j])
            else:
                f=list(featureVector("/home/codit/Documents/Time Series JSON/daily/"+j).values())
                if forcastability_score_cosine_with_rep(f)>=i:
                    visited["daily"+j]=predict(pd.read_json("/home/codit/Documents/Time Series JSON/daily/"+j))
                    rmse.append(visited["daily"+j])
        for j in weekly:
            if "weekly"+j in list(visited.keys()) and visited["weekly"+j]>=i:
                rmse.append(visited["weekly"+j])
            else:
                f=list(featureVector("/home/codit/Documents/Time Series JSON/weekly/"+j).values())
                if forcastability_score_cosine_with_rep(f)>=i:
                    visited["weekly"+j]=predict(pd.read_json("/home/codit/Documents/Time Series JSON/weekly/"+j))
                    rmse.append(visited["weekly"+j])
                    
        for j in monthly:
            if "monthly"+j in list(visited.keys()) and visited["monthly"+j]>=i:
                rmse.append(visited["monthly"+j])
            else:
                f=list(featureVector("/home/codit/Documents/Time Series JSON/monthly/"+j).values())
                if forcastability_score_cosine_with_rep(f)>=i:
                    visited["monthly"+j]=predict(pd.read_json("/home/codit/Documents/Time Series JSON/monthly/"+j))
                    rmse.append(visited["monthly"+j])
                    
        
        if len(rmse)!=0:
            mean=sum(rmse)/len(rmse)
            print(mean)
            if minMAPEGen>mean:
                minMAPEGen=mean
                threshold=i
                
    return threshold
                
    
def predict(data):
    
    data=data.rename(columns={'point_timestamp':'ds','point_value':'y'})
    data['ds'] = pd.to_datetime(data['ds'])
    data['ds'] = data['ds'].dt.tz_localize(None)
    model=Prophet()
    #have 80 percent of the data for training
    train = data.iloc[:int(0.8*len(data))]
    train.replace(0, train['y'].mean(), inplace=True)
    test = data.iloc[int(0.8*len(data)):]
    test.replace(0,train['y'].mean(),inplace=True)
    model.fit(train)
    forecast=model.predict(test)
    return root_mean_squared_error(test['y'],forecast['yhat'].tail(len(test)))


In [ ]:
import json
result=forecastabilityThreshold(sampleforecastScores)
threshold = {"threshold":result}
with open('threshold.json', 'w') as f:
    json.dump(threshold, f)
    

